In [136]:
import re
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from textwrap import indent

In [137]:
# Comment these out and appropriate cells according to the experiment evaluated
LOG_DIR = Path("logs/runs/exp1")
# LOG_DIR = Path("logs/runs/exp2")
# LOG_DIR = Path("logs/runs/exp3")

PATTERNS = {
    "wall": re.compile(r"CORE::TIME::WALL\s+([0-9.]+)"),
    "scatter": re.compile(r"CORE::TIME::SCATTER\s+([0-9.]+)"),
    "gather": re.compile(r"CORE::TIME::GATHER\s+([0-9.]+)"),
    "phases": re.compile(r"CORE::PHASES\s+([0-9]+)"),
    "edges_streamed": re.compile(r"CORE::BYTES::EDGES_STREAMED\s+([0-9]+)"),
    "updates_in": re.compile(r"CORE::BYTES::UPDATES_IN\s+([0-9]+)"),
    "updates_out": re.compile(r"CORE::BYTES::UPDATES_OUT\s+([0-9]+)")
}

def parse_log(path: Path) -> dict:
    text = path.read_text()
    out = {"log": path.name}
    for key, pat in PATTERNS.items():
        m = pat.findall(text)
        if not m:
            out[key] = None
        else:
            out[key] = sum(float(x) for x in m)
    return out


In [138]:
# graph = "cit-Patents"
INPUT_GRAPH = "*"
ALGORITHM = "*"
RUN_ID = "*"

PATTERN = f"{INPUT_GRAPH}-{ALGORITHM}-{RUN_ID}.log"
# PATTERN = f"soc-LiveJournal1-*k32-r0.log"

logs = sorted(LOG_DIR.glob(PATTERN))
records = [parse_log(p) for p in logs]
df = pd.DataFrame(records)

EDGE_REC_BYTES = 12  # type=1 COMPACT (@IIf)
UPDATE_REC_BYTES = 8

df["# iters"] = df["phases"].astype("Int64")

df["streaming_time"] = df["scatter"] + df["gather"]
df["ratio"] = df["wall"] / df["streaming_time"]

df["edges_streamed_cnt"] = df["edges_streamed"] / EDGE_REC_BYTES
df["updates_out_cnt"] = df["updates_out"] / UPDATE_REC_BYTES
df["wasted %"] = 100 * (1 - (df["updates_out_cnt"] / df["edges_streamed_cnt"]))

df

,log,wall,scatter,gather,phases,edges_streamed,updates_in,updates_out,# iters,streaming_time,ratio,edges_streamed_cnt,updates_out_cnt,wasted %
0,amazon0601-cond-r0.log,3.80383,0.286000,1.00000,2.0,8.129731e+07,0.000000e+00,0.000000e+00,2,1.286000,2.957877,6.774776e+06,0.000000e+00,100.000000
1,amazon0601-cond-r1.log,3.75545,0.254001,0.00000,2.0,8.129731e+07,0.000000e+00,0.000000e+00,2,0.254001,14.785178,6.774776e+06,0.000000e+00,100.000000
2,amazon0601-cond-r2.log,3.79172,0.283001,1.00000,2.0,8.129731e+07,0.000000e+00,0.000000e+00,2,1.283001,2.955352,6.774776e+06,0.000000e+00,100.000000
3,amazon0601-mcst-r0.log,11.46530,NaN,NaN,7.0,NaN,NaN,NaN,7,NaN,NaN,NaN,NaN,NaN
4,amazon0601-mcst-r1.log,12.24670,NaN,NaN,7.0,NaN,NaN,NaN,7,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,soc-LiveJournal1-sssp-r1.log,15.70930,7.293520,3.44593,51.0,3.587822e+10,8.215034e+09,8.215034e+09,51,10.739450,1.462766,2.989851e+09,1.026879e+09,65.654503
86,soc-LiveJournal1-sssp-r2.log,14.72890,6.992010,3.11969,51.0,3.587822e+10,8.215034e+09,8.215034e+09,51,10.111700,1.456620,2.989851e+09,1.026879e+09,65.654503
87,soc-LiveJournal1-wcc-r0.log,12.54590,6.970640,1.67926,13.0,1.792734e+10,5.103943e+09,5.103943e+09,13,8.649900,1.450410,1.493945e+09,6.379928e+08,57.294752
88,soc-LiveJournal1-wcc-r1.log,12.15710,6.693010,1.58953,13.0,1.792734e+10,5.103943e+09,5.103943e+09,13,8.282540,1.467799,1.493945e+09,6.379928e+08,57.294752


In [139]:
# Experiment 1b
df_wcc = df[df["log"].str.contains("-wcc-")]
df_wcc

,log,wall,scatter,gather,phases,edges_streamed,updates_in,updates_out,# iters,streaming_time,ratio,edges_streamed_cnt,updates_out_cnt,wasted %
21,amazon0601-wcc-r0.log,6.00316,1.35201,0.646716,19.0,1.432886e+09,3.501340e+08,3.501340e+08,19,1.998726,3.003493,1.194071e+08,4.376675e+07,63.346620
22,amazon0601-wcc-r1.log,6.24956,1.53301,0.683528,19.0,1.432886e+09,3.501340e+08,3.501340e+08,19,2.216538,2.819514,1.194071e+08,4.376675e+07,63.346620
23,amazon0601-wcc-r2.log,5.30511,1.02100,0.458833,19.0,1.432886e+09,3.501340e+08,3.501340e+08,19,1.479833,3.584938,1.194071e+08,4.376675e+07,63.346620
45,cit-Patents-wcc-r0.log,7.72196,2.46100,1.183990,21.0,6.752079e+09,2.173422e+09,2.173422e+09,21,3.644990,2.118513,5.626733e+08,2.716777e+08,51.716611
46,cit-Patents-wcc-r1.log,8.61865,3.15201,1.313230,21.0,6.752079e+09,2.173422e+09,2.173422e+09,21,4.465240,1.930165,5.626733e+08,2.716777e+08,51.716611
47,cit-Patents-wcc-r2.log,7.66075,2.45900,1.153800,21.0,6.752079e+09,2.173422e+09,2.173422e+09,21,3.612800,2.120447,5.626733e+08,2.716777e+08,51.716611
63,dimacs-usa-wcc-r0.log,979.70300,666.81200,189.482000,6263.0,8.549308e+12,1.364612e+11,1.364612e+11,6263,856.294000,1.144120,7.124423e+11,1.705765e+10,97.605750
64,dimacs-usa-wcc-r1.log,984.21500,668.74800,191.034000,6263.0,8.549308e+12,1.364612e+11,1.364612e+11,6263,859.782000,1.144726,7.124423e+11,1.705765e+10,97.605750
65,dimacs-usa-wcc-r2.log,975.70000,665.75600,188.110000,6263.0,8.549308e+12,1.364612e+11,1.364612e+11,6263,853.866000,1.142685,7.124423e+11,1.705765e+10,97.605750
87,soc-LiveJournal1-wcc-r0.log,12.54590,6.97064,1.679260,13.0,1.792734e+10,5.103943e+09,5.103943e+09,13,8.649900,1.450410,1.493945e+09,6.379928e+08,57.294752


In [140]:
# Experiment 1

GRAPH_NAMES = [
    "amazon0601",
    "cit-Patents",
    "dimacs-usa",
    "soc-LiveJournal1",
]

RUN_RE = re.compile(r"-r(?P<run>\d+)\.log$")

def split_log_name(filename: str) -> dict:
    # find graph by prefix match (graph names may include '-')
    graph = None
    for g in GRAPH_NAMES:
        prefix = g + "-"
        if filename.startswith(prefix):
            graph = g
            rest = filename[len(prefix):]  # e.g. "wcc-r0.log"
            break
    if graph is None:
        raise ValueError(f"Unknown graph prefix in filename: {filename}")

    m = RUN_RE.search(filename)
    if not m:
        raise ValueError(f"Cannot parse run id from filename: {filename}")
    run = int(m.group("run"))

    alg = rest[: rest.rfind(f"-r{run}.log")]

    return {"graph": graph, "algorithm": alg, "run": run}

meta = df["log"].apply(split_log_name).apply(pd.Series)
df = pd.concat([df, meta], axis=1)

df["run"] = df["run"].astype(int)

METRICS = [
    "wall",
    "scatter",
    "gather",
    "streaming_time",
    "ratio",
    "wasted %",
    "# iters",
    "edges_streamed_cnt",
    "updates_out_cnt",
]

summary = (
    df.groupby(["graph", "algorithm"], dropna=False)[METRICS]
      .agg(["count", "mean", "std"])
      .reset_index()
)

summary

graph algorithm  wall                       scatter  \
                               count        mean       std   count   
0         amazon0601      cond     3    3.783667  0.025175       3   
1         amazon0601      mcst     3   12.579933  1.313349       0   
2         amazon0601       mis     3   11.685867  1.544613       3   
3         amazon0601  pagerank     3    6.010277  0.430817       3   
4         amazon0601       scc     3   11.447267  1.384514       0   
5         amazon0601      spmv     3    3.795517  0.067705       3   
6         amazon0601      sssp     3    9.670260  0.855998       3   
7         amazon0601       wcc     3    5.852610  0.489893       3   
8        cit-Patents      cond     3    4.810693  0.418038       3   
9        cit-Patents      mcst     3   18.995867  0.623902       0   
10       cit-Patents       mis     3    7.440327  0.819849       3   
11       cit-Patents  pagerank     3    8.023987  0.469046       3   
12       cit-Patents       scc     3    6.505893  0.464375       0   
13       cit-Patents      spmv     3    4.234623  0.061048       3   
14       cit-Patents      sssp     3    4.181373  0.054155       3   
15       cit-Patents       wcc     3    8.000453  0.536248       3   
16        dimacs-usa      cond     3    7.397260  0.303120       3   
17        dimacs-usa      mcst     3   38.375500  2.069254       0   
18        dimacs-usa       mis     3   16.018933  0.327388       3   
19        dimacs-usa  pagerank     3   14.084600  0.486641       3   
20        dimacs-usa      spmv     3    6.044853  0.057491       3   
21        dimacs-usa       wcc     3  979.872667  4.260035       3   
22  soc-LiveJournal1      cond     3    8.567143  0.257312       3   
23  soc-LiveJournal1      mcst     3   85.410000  5.493114       0   
24  soc-LiveJournal1       mis     3   22.196600  0.649912       3   
25  soc-LiveJournal1  pagerank     3   13.298033  0.492236       3   
26  soc-LiveJournal1       scc     3   13.908767  0.778339       0   
27  soc-LiveJournal1      spmv     3    6.823270  0.596873       3   
28  soc-LiveJournal1      sssp     3   14.992033  0.628469       3   
29  soc-LiveJournal1       wcc     3   12.598400  0.469755       3   

                         gather              ... wasted % # iters            \
          mean       std  count        mean  ...      std   count      mean   
0     0.274334  0.017673      3    0.666667  ...      0.0       3       2.0   
1          NaN       NaN      0         NaN  ...      NaN       3  6.666667   
2     4.020333  0.749543      3    1.893843  ...      0.0       3      68.0   
3     1.246330  0.210029      3    0.605090  ...      0.0       3      22.0   
4          NaN       NaN      0         NaN  ...      NaN       0      <NA>   
5     0.226002  0.037510      3    0.034668  ...      0.0       3       2.0   
6     3.177733  0.426682      3    1.625340  ...      0.0       3      64.0   
7     1.302007  0.259642      3    0.596359  ...      0.0       3      19.0   
8     1.264333  0.437710      3    1.333333  ...      0.0       3       2.0   
9          NaN       NaN      0         NaN  ...      NaN       3       6.0   
10    2.353997  0.489374      3    0.623389  ...      0.0       3      15.0   
11    2.068670  0.231165      3    1.333290  ...      0.0       3      22.0   
12         NaN       NaN      0         NaN  ...      NaN       0      <NA>   
13    0.564001  0.036511      3    0.069146  ...      0.0       3       2.0   
14    0.532001  0.027055      3    0.333333  ...      0.0       3       2.0   
15    2.690670  0.399533      3    1.217007  ...      0.0       3      21.0   
16    3.829000  0.301209      3    0.333333  ...      0.0       3       2.0   
17         NaN       NaN      0         NaN  ...      NaN       3  9.333333   
18    8.028000  0.345446      3    2.357397  ...      0.0       3      31.0   
19    4.678000  0.213107      3    3.704663  ...      0.0       3      22.0   
20    1.922003  0.022649      3    0

In [141]:
summary_wcc = summary[summary["algorithm"].str.contains("wcc")]
with pd.option_context(
    "display.max_rows", None,
    "display.max_columns", None,
    "display.max_colwidth", None,
):
    display(summary_wcc)

graph algorithm  wall                       scatter  \
                               count        mean       std   count   
7         amazon0601       wcc     3    5.852610  0.489893       3   
15       cit-Patents       wcc     3    8.000453  0.536248       3   
21        dimacs-usa       wcc     3  979.872667  4.260035       3   
29  soc-LiveJournal1       wcc     3   12.598400  0.469755       3   

                         gather                       streaming_time  \
          mean       std  count        mean       std          count   
7     1.302007  0.259642      3    0.596359  0.120515              3   
15    2.690670  0.399533      3    1.217007  0.084688              3   
21  667.105333  1.517415      3  189.542000  1.462923              3   
29    7.109220  0.500114      3    1.611767  0.059573              3   

                         ratio                     wasted %                  \
          mean       std count      mean       std    count       mean  std   
7     1.898366  0.378468     3  3.135982  0.399542        3  63.346620  0.0   
15    3.907677  0.483132     3  2.056375  0.109305        3  51.716611  0.0   
21  856.647333  2.973785     3  1.143844  0.001048        3  97.605750  0.0   
29    8.720987  0.477971     3  1.445523  0.025079        3  57.294752  0.0   

   # iters              edges_streamed_cnt                    updates_out_cnt  \
     count    mean  std              count          mean  std           count   
7        3    19.0  0.0                  3  1.194071e+08  0.0               3   
15       3    21.0  0.0                  3  5.626733e+08  0.0               3   
21       3  6263.0  0.0                  3  7.124423e+11  0.0               3   
29       3    13.0  0.0                  3  1.493945e+09  0.0               3   

                       
            mean  std  
7   4.376675e+07  0.0  
15  2.716777e+08  0.0  
21  1.705765e+10  0.0  
29  6.379928e+08  0.0

In [142]:
# Experiment 1 (pretty)

def mean_pm_sd(s: pd.Series) -> str:
    s = s.dropna()
    if len(s) == 0:
        return ""
    if len(s) == 1:
        return f"{s.iloc[0]:.4g}"
    return f"{s.mean():.4g} ± {s.std(ddof=1):.2g}"

pretty = (
    df.groupby(["graph", "algorithm"])
      .agg(
          n=("wall", lambda x: x.notna().sum()),
          wall_s=("wall", mean_pm_sd),
          streaming_s=("streaming_time", mean_pm_sd),
          wasted_s=("wasted %", mean_pm_sd),
          iters_s=("# iters", mean_pm_sd),
          ratio_s=("ratio", mean_pm_sd),
      )
      .reset_index()
      .sort_values(["graph", "algorithm"])
)

pretty


,graph,algorithm,n,wall_s,streaming_s,wasted_s,iters_s,ratio_s
0,amazon0601,cond,3,3.784 ± 0.025,0.941 ± 0.59,100 ± 0,2 ± 0,6.899 ± 6.8
1,amazon0601,mcst,3,12.58 ± 1.3,,,6.667 ± 0.58,
2,amazon0601,mis,3,11.69 ± 1.5,5.914 ± 1.1,91.77 ± 0,68 ± 0,1.989 ± 0.11
3,amazon0601,pagerank,3,6.01 ± 0.43,1.851 ± 0.32,4.545 ± 0,22 ± 0,3.281 ± 0.3
4,amazon0601,scc,3,11.45 ± 1.4,,,,
5,amazon0601,spmv,3,3.796 ± 0.068,0.2607 ± 0.047,0 ± 0,2 ± 0,14.84 ± 2.3
6,amazon0601,sssp,3,9.67 ± 0.86,4.803 ± 0.67,77.47 ± 0,64 ± 0,2.022 ± 0.095
7,amazon0601,wcc,3,5.853 ± 0.49,1.898 ± 0.38,63.35 ± 0,19 ± 0,3.136 ± 0.4
8,cit-Patents,cond,3,4.811 ± 0.42,2.598 ± 0.62,100 ± 0,2 ± 0,1.917 ± 0.42
9,cit-Patents,mcst,3,19 ± 0.62,,,6 ± 0,


In [143]:
# Experiment 1: Comparison with Paper

paper_memory = {
    ("amazon0601", "wcc"): 0.61,
    ("amazon0601", "scc"): 1.12,
    ("amazon0601", "sssp"): 0.83,
    ("amazon0601", "mcst"): 0.37,
    ("amazon0601", "mis"): 3.31,
    ("amazon0601", "cond"): 0.07,
    ("amazon0601", "spmv"): 0.09,
    ("amazon0601", "pagerank"): 0.25,

    ("cit-Patents", "wcc"): 2.98,
    ("cit-Patents", "scc"): 0.69,
    ("cit-Patents", "sssp"): 0.29,
    ("cit-Patents", "mcst"): 2.35,
    ("cit-Patents", "mis"): 3.72,
    ("cit-Patents", "cond"): 0.19,
    ("cit-Patents", "spmv"): 0.19,
    ("cit-Patents", "pagerank"): 0.74,

    ("soc-LiveJournal1", "wcc"): 7.22,
    ("soc-LiveJournal1", "scc"): 11.12,
    ("soc-LiveJournal1", "sssp"): 9.60,
    ("soc-LiveJournal1", "mcst"): 7.66,
    ("soc-LiveJournal1", "mis"): 15.54,
    ("soc-LiveJournal1", "cond"): 0.78,
    ("soc-LiveJournal1", "spmv"): 0.74,
    ("soc-LiveJournal1", "pagerank"): 2.90,

    ("dimacs-usa", "wcc"): 372.0,
    ("dimacs-usa", "scc"): 594.0,
    ("dimacs-usa", "sssp"): 2312.0,
    ("dimacs-usa", "mcst"): 4.68,
    ("dimacs-usa", "mis"): 9.60,
    ("dimacs-usa", "cond"): 0.26,
    ("dimacs-usa", "spmv"): 0.65,
    ("dimacs-usa", "pagerank"): 2.58,
}

paper_df = (
    pd.Series(paper_memory, name="paper_wall_s")
      .rename_axis(["graph", "algorithm"])
      .reset_index()
)

agg = (
    df.groupby(["graph", "algorithm"])
      .agg(
          n=("wall", "count"),
          wall_mean=("wall", "mean"),
          wall_std=("wall", lambda x: x.std(ddof=1)),
      )
      .reset_index()
)

agg["ours_wall_s"] = (
    agg["wall_mean"].map(lambda x: f"{x:.2f}")
    + " ± "
    + agg["wall_std"].fillna(0).map(lambda x: f"{x:.2f}")
)

compare = (
    paper_df
    .merge(agg, on=["graph", "algorithm"], how="left")
    .sort_values(["graph", "algorithm"])
    .reset_index(drop=True)
)

compare["factor_vs_paper"] = compare["wall_mean"] / compare["paper_wall_s"]
compare["factor_vs_paper"] = compare["factor_vs_paper"].round(2)

compare


,graph,algorithm,paper_wall_s,n,wall_mean,wall_std,ours_wall_s,factor_vs_paper
0,amazon0601,cond,0.07,3.0,3.783667,0.025175,3.78 ± 0.03,54.05
1,amazon0601,mcst,0.37,3.0,12.579933,1.313349,12.58 ± 1.31,34.00
2,amazon0601,mis,3.31,3.0,11.685867,1.544613,11.69 ± 1.54,3.53
3,amazon0601,pagerank,0.25,3.0,6.010277,0.430817,6.01 ± 0.43,24.04
4,amazon0601,scc,1.12,3.0,11.447267,1.384514,11.45 ± 1.38,10.22
5,amazon0601,spmv,0.09,3.0,3.795517,0.067705,3.80 ± 0.07,42.17
6,amazon0601,sssp,0.83,3.0,9.670260,0.855998,9.67 ± 0.86,11.65
7,amazon0601,wcc,0.61,3.0,5.852610,0.489893,5.85 ± 0.49,9.59
8,cit-Patents,cond,0.19,3.0,4.810693,0.418038,4.81 ± 0.42,25.32
9,cit-Patents,mcst,2.35,3.0,18.995867,0.623902,19.00 ± 0.62,8.08


In [144]:
# # Experiment 2
# pattern = (
#     r"-"                             # dash before algorithm
#     r"(?P<algorithm>[a-zA-Z0-9_]+)"  # algorithm name
#     r"-mem(?P<memory>\d+[a-zA-Z])"   # memory (e.g. 1g, 8g)
#     r"-r(?P<run>\d+)"                # run id
# )

# df[["algorithm", "memory", "run"]] = (
#     df["log"]
#     .str.extract(pattern)
# )

# def mem_to_mb(s: str) -> int:
#     s = s.lower()
#     if s.endswith("g"):
#         return int(s[:-1]) * 1024
#     if s.endswith("m"):
#         return int(s[:-1])
#     raise ValueError(f"Unknown memory format: {s}")

# df["memory"] = df["memory"].apply(mem_to_mb)

# front = ["algorithm", "memory", "run", "wall"]
# df = df[front + [c for c in df.columns if c not in front]]

# with pd.option_context(
#     "display.max_rows", None,
#     "display.max_columns", None,
#     "display.max_colwidth", None,
# ):
#     display(df.sort_values(by=["algorithm", "memory"]))

In [145]:
# # Experiment 2: Plots

# # Ensure correct dtypes
# df["run"] = df["run"].astype(int)
# df["memory"] = df["memory"].astype(int)

# # Aggregate
# df = df.sort_values(["algorithm", "memory", "run"])

# agg = (
#     df.groupby(["algorithm", "memory"], as_index=False)
#       .agg(
#           wall_mean=("wall", "mean"),
#           wall_std=("wall", "std"),
#           n=("wall", "size"),
#       )
# )

# plt.figure(figsize=(9, 5))

# for alg, sub in agg.groupby("algorithm"):
#     sub = sub.sort_values("memory")

#     # Mean line (matplotlib assigns a color; we reuse it)
#     (line,) = plt.plot(
#         sub["memory"],
#         sub["wall_mean"],
#         marker="o",
#         label=alg,
#     )
#     color = line.get_color()

#     # Shaded +/- 1 std when multiple runs exist
#     if sub["wall_std"].notna().any() and (sub["n"] > 1).any():
#         plt.fill_between(
#             sub["memory"],
#             sub["wall_mean"] - sub["wall_std"],
#             sub["wall_mean"] + sub["wall_std"],
#             alpha=0.15,
#             color=color,  # match the line color
#         )

#     # Per-run dots in the same color as the line
#     runs = df[df["algorithm"] == alg].sort_values("memory")
#     plt.scatter(
#         runs["memory"],
#         runs["wall"],
#         s=10,
#         alpha=0.7,
#         color=color,
#         edgecolors="none",
#     )

# plt.xscale("log", base=2)
# mems = sorted(df["memory"].unique())
# plt.xticks(mems, mems)
# plt.xlabel("Memory (MB)")
# plt.ylabel("Runtime (s)")
# plt.title("Runtime vs memory size")
# plt.grid(True, which="both", linestyle="--", alpha=0.3)
# plt.legend(ncol=2, fontsize=9)
# plt.tight_layout()
# plt.savefig("./exp2.png", dpi=200)
# plt.show()



In [146]:
# # Experiment 3
# pattern = (
#     r"-"                             # dash before algorithm
#     r"(?P<algorithm>[a-zA-Z0-9_]+)"  # algorithm name
#     r"-th(?P<threads>\d+)"           # threads
#     r"-mem(?P<memory>\d+[a-zA-Z])"   # memory (e.g. 1g, 8g)
#     r"-r(?P<run>\d+)"                # run id
# )

# df[["algorithm", "threads", "memory", "run"]] = (
#     df["log"]
#     .str.extract(pattern)
# )

# def mem_to_mb(s: str) -> int:
#     s = s.lower()
#     if s.endswith("g"):
#         return int(s[:-1]) * 1024
#     if s.endswith("m"):
#         return int(s[:-1])
#     raise ValueError(f"Unknown memory format: {s}")

# df["memory"] = df["memory"].apply(mem_to_mb)

# df["threads"] = df["threads"].astype(int)

# front = ["algorithm", "memory", "threads", "run", "wall"]
# df = df[front + [c for c in df.columns if c not in front]]

# df.sort_values(by=["algorithm", "memory", "threads"])

In [147]:
# # Experiment 3 contd.
# df = df.sort_values(["algorithm", "memory", "threads", "run"])

# agg = (df.groupby(["algorithm", "memory", "threads"], as_index=False)
#          .agg(wall_mean=("wall", "mean"),
#               wall_std=("wall", "std"),
#               n=("wall", "size")))

# plt.figure(figsize=(9, 5))

# for (alg, mem), sub in agg.groupby(["algorithm", "memory"]):
#     sub = sub.sort_values("threads")
#     # label = f"{alg} @ {mem} MB"
#     label = f"{alg}"
#     plt.plot(sub["threads"], sub["wall_mean"], marker="o", label=label)

#     # Shaded +/- 1 std with multiple runs
#     if sub["wall_std"].notna().any() and (sub["n"] > 1).any():
#         plt.fill_between(
#             sub["threads"],
#             sub["wall_mean"] - sub["wall_std"],
#             sub["wall_mean"] + sub["wall_std"],
#             alpha=0.15
#         )

# plt.xscale("log", base=2)
# plt.xticks(sorted(df["threads"].unique()), sorted(df["threads"].unique()))
# plt.xlabel("Threads")
# plt.ylabel("Wall time (s)")
# plt.title("Runtime vs number of threads")
# plt.grid(True, which="both", linestyle="--", alpha=0.3)
# plt.legend(ncol=2, fontsize=9)
# plt.tight_layout()
# plt.savefig("./exp3.png")
# plt.show()


In [148]:
# # Experiment 3 contd. — with per-run dots

# df = df.sort_values(["algorithm", "memory", "threads", "run"])

# agg = (
#     df.groupby(["algorithm", "memory", "threads"], as_index=False)
#       .agg(
#           wall_mean=("wall", "mean"),
#           wall_std=("wall", "std"),
#           n=("wall", "size"),
#       )
# )

# plt.figure(figsize=(9, 5))

# for (alg, mem), sub in agg.groupby(["algorithm", "memory"]):
#     sub = sub.sort_values("threads")
#     # label = f"{alg} @ {mem} MB"
#     label = f"{alg}"

#     # Mean line (capture color)
#     (line,) = plt.plot(
#         sub["threads"],
#         sub["wall_mean"],
#         marker="o",
#         label=label,
#     )
#     color = line.get_color()

#     # Shaded +/- 1 std
#     if sub["wall_std"].notna().any() and (sub["n"] > 1).any():
#         plt.fill_between(
#             sub["threads"],
#             sub["wall_mean"] - sub["wall_std"],
#             sub["wall_mean"] + sub["wall_std"],
#             alpha=0.15,
#             color=color,
#         )

#     # Per-run dots (same color as the line)
#     runs = df[
#         (df["algorithm"] == alg) &
#         (df["memory"] == mem)
#     ].sort_values("threads")

#     plt.scatter(
#         runs["threads"],
#         runs["wall"],
#         s=10,
#         alpha=0.7,
#         color=color,
#         edgecolors="none",
#     )

# plt.xscale("log", base=2)
# plt.xticks(
#     sorted(df["threads"].unique()),
#     sorted(df["threads"].unique())
# )
# plt.xlabel("# Threads")
# plt.ylabel("Runtime (s)")
# plt.title("Runtime vs number of threads")
# plt.grid(True, which="both", linestyle="--", alpha=0.3)
# plt.legend(ncol=2, fontsize=9)
# plt.tight_layout()
# plt.savefig("./exp3.png", dpi=200)
# plt.show()
